<a href="https://colab.research.google.com/github/christianasb/Python-Capstone-DataCamp-Mini-Projects/blob/main/car-insurance-modeling/Modeling_Car_Insurance_Claim_Outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`)

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code |
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client |
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

## Importing the Data

In [20]:
# Import required libraries
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
import requests
from io import StringIO

#read in the data from a Google Drive shareable link
orig_url="https://drive.google.com/file/d/1E3lcnBCn2Pn08QBFIj5HK0soPQmMntAB/view?usp=sharing"

file_id = orig_url.split('/')[-2] #extract file ID from url
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id

#download and convert CSV content into a readable format
url = requests.get(dwn_url).text
csv_raw = StringIO(url)

#read the data into a pandas DataFrame
df_insurance = pd.read_csv(csv_raw)

#preview the dataset
df_insurance.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


## Examining and Cleaning the Data

In [21]:
#examining car_insurance data
print(df_insurance.describe())
print(df_insurance.info())

                  id           age        gender  credit_score  \
count   10000.000000  10000.000000  10000.000000   9018.000000   
mean   500521.906800      1.489500      0.499000      0.515813   
std    290030.768758      1.025278      0.500024      0.137688   
min       101.000000      0.000000      0.000000      0.053358   
25%    249638.500000      1.000000      0.000000      0.417191   
50%    501777.000000      1.000000      0.000000      0.525033   
75%    753974.500000      2.000000      1.000000      0.618312   
max    999976.000000      3.000000      1.000000      0.960819   

       vehicle_ownership       married      children   postal_code  \
count       10000.000000  10000.000000  10000.000000  10000.000000   
mean            0.697000      0.498200      0.688800  19864.548400   
std             0.459578      0.500022      0.463008  18915.613855   
min             0.000000      0.000000      0.000000  10238.000000   
25%             0.000000      0.000000      0.000000  1

In [22]:
#filling in missing values with mean
df_insurance = df_insurance.fillna(df_insurance.mean(numeric_only=True))

## Creating Variables for Modeling and Storing Results

In [23]:
#create an empty list to store each model object
models = []

#create a features variable of all columns except for "outcome" and "id"
features = df_insurance.drop(columns=["id", "outcome"]).columns


## Building and Storing the Models

In [24]:
#building one logistic regression model per feature and saving results to a list

#loop through each feature to build a separate model predicting 'outcome'
for col in features:
    formula = f"outcome ~ {col}"                           #defines the formula
    model = logit(formula, data=df_insurance).fit(disp=0)  #fits the logistic regression model
    models.append(model)                                   #appends the fitted model to the list
    print(f"{col}")

age
gender
driving_experience
education
income
credit_score
vehicle_ownership
vehicle_year
married
children
postal_code
annual_mileage
vehicle_type
speeding_violations
duis
past_accidents


## Measuring Model Performance

In [25]:
#create a list to store model accuracies
accuracies = []

#loop through each model stored in the models list
for i in range(len(models)):
    model = models[i]

    #creating a confusion matrix and storing individual metrics
    conf_matrix = model.pred_table()
    tn = conf_matrix[0, 0]  #true negatives
    fp = conf_matrix[0, 1]  #false positive
    fn = conf_matrix[1, 0]  #false negatives
    tp = conf_matrix[1, 1]  #true positives

    #calculating accuracy
    accuracy = (tn + tp) / (tn + fp + fn + tp)

    #storing each accuracy in the accuracies list
    accuracies.append(accuracy)

    #printing each feature & its corresponding accuracy
    print(f"Feature: {features[i]} | Accuracy: {accuracy:.4f}")

Feature: age | Accuracy: 0.7747
Feature: gender | Accuracy: 0.6867
Feature: driving_experience | Accuracy: 0.7771
Feature: education | Accuracy: 0.6867
Feature: income | Accuracy: 0.7425
Feature: credit_score | Accuracy: 0.7054
Feature: vehicle_ownership | Accuracy: 0.7351
Feature: vehicle_year | Accuracy: 0.6867
Feature: married | Accuracy: 0.6867
Feature: children | Accuracy: 0.6867
Feature: postal_code | Accuracy: 0.6867
Feature: annual_mileage | Accuracy: 0.6904
Feature: vehicle_type | Accuracy: 0.6867
Feature: speeding_violations | Accuracy: 0.6867
Feature: duis | Accuracy: 0.6867
Feature: past_accidents | Accuracy: 0.6867


## Finding the Best Performing Model

In [26]:
#find the index of the model with highest accuracy
best_index = accuracies.index(max(accuracies))

#linking the highest accuracy to the corresponding feature name and its accuracy score
best_feature = features[best_index]
best_accuracy = accuracies[best_index]

#creating a summary DataFrame with the best feature and accuracy
best_feature_df = pd.DataFrame({
    "best_feature": [best_feature],
    "best_accuracy": [best_accuracy]
}, index=[0])

#printing the result
print(best_feature_df)

         best_feature  best_accuracy
0  driving_experience         0.7771


## Interpretation & Recommendations for Car Insurance Claim Risk Modeling

Based on the analysis of customer data provided by On the Road Car Insurance, I identified the single best feature to predict whether a customer will file a claim during the policy period. This insight allows the company to launch a simple, interpretable model with minimal infrastructure requirements.

**Key Findings:**
- **Top predictive feature is driving experience.** This feature alone produced the highest **model accuracy of 77.71%**, outperforming all other variables in predicting whether a customer would make a claim.

- **Other feature performance:**
Features like age, income, and credit_score showed moderate predictive power, while variables such as gender, education, and vehicle_type provided little to no improvement in model accuracy.

**Recommended Actions:**
1. **Deploy a single-feature model using driving_experience:**  
   Begin using a simple logistic regression model based only on this feature to assess claim risk. It’s easy to explain, quick to implement, and offers reasonable predictive performance. This can be scaled later once infrastructure matures.

2. **Incorporate `driving_experience` into premium pricing strategies.**  
   Use model evidence to support and potentially recalibrate premium differentials based on driving experience. The data shows strong justification for higher rates among less experienced drivers, which may warrant adjustment of existing pricing tiers.

3. **Monitor model performance over time.**
Track claim outcomes for customers scored with the driving_experience model to assess real-world accuracy. This ongoing validation will support data-driven decision-making and model refinement when infrastructure improves.

4. **Target high-risk segments with tailored risk-reduction programs.**  
   While onboarding support for new drivers may already exist, this model can help prioritize who receives outreach, enabling more personalized interventions (e.g., offering safe-driving telematics or adjusting underwriting thresholds).

---